## Preprocessing

In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [18]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [19]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type = application_df['APPLICATION_TYPE'].value_counts()
application_type


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [20]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_type = application_df['APPLICATION_TYPE'].value_counts()
application_type = application_type[application_type < 500].index.tolist()
application_types_to_replace = application_type


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [21]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [22]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# to see which ones will be replaced
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count = classification_count[classification_count > 1] 
classification_count


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [23]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count = classification_count[classification_count < 100].index.tolist()   
classifications_to_replace = classification_count


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 
                                                        'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 
                                                        'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], drop_first=True)

application_df = application_df.astype(int)
application_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [25]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns=['IS_SUCCESSFUL']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               5504      
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13,825
Trainable params: 13,825
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
644/644 [==============================] - 2s 2ms/step - loss: 0.5704 - accuracy: 0.7240 - val_loss: 0.5678 - val_accuracy: 0.7215
Epoch 2/100
644/644 [==============================] - 1s 2ms/step - loss: 0.5532 - accuracy: 0.7319 - val_loss: 0.5652 - val_accuracy: 0.7176
Epoch 3/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5504 - accuracy: 0.7300 - val_loss: 0.5710 - val_accuracy: 0.7213
Epoch 4/100
644/644 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7339 - val_loss: 0.5618 - val_accuracy: 0.7234
Epoch 5/100
644/644 [==============================] - 1s 2ms/step - loss: 0.5458 - accuracy: 0.7352 - val_loss: 0.5587 - val_accuracy: 0.7236
Epoch 6/100
644/644 [==============================] - 1s 2ms/step - loss: 0.5454 - accuracy: 0.7343 - val_loss: 0.5593 - val_accuracy: 0.7234
Epoch 7/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7359 - val_loss: 0.5599 - val_accuracy: 0.7267

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5769 - accuracy: 0.7276 - 313ms/epoch - 1ms/step
Loss: 0.5768588781356812, Accuracy: 0.727580189704895


In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

## Optimize the Model

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=256, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 256)               11008     
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 52,225
Trainable params: 52,225
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model with the Adam optimizer and binary crossentropy loss function
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [33]:
# Train the model with more epochs and a batch size of 32 for better performance
fit_model = nn.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_data=(X_test_scaled, y_test))


Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5672 - accuracy: 0.7226 - val_loss: 0.5596 - val_accuracy: 0.7229
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7309 - val_loss: 0.5561 - val_accuracy: 0.7255
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7319 - val_loss: 0.5580 - val_accuracy: 0.7265
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7331 - val_loss: 0.5560 - val_accuracy: 0.7262
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7331 - val_loss: 0.5522 - val_accuracy: 0.7252
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7348 - val_loss: 0.5532 - val_accuracy: 0.7269
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7343 - val_loss: 0.5515 - val_accuracy: 0.7282

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8994 - accuracy: 0.7256 - 395ms/epoch - 1ms/step
Loss: 0.8993940353393555, Accuracy: 0.7255976796150208


In [35]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')
